In [128]:
import numpy as np

class GenomeTemplate:
    def __init__(self, input_len, output_len):
        self.connections = []
        self.input_len = input_len
        self.output_len = output_len
        self.innovation_num = 0
        self.node_num = self.input_len + self.output_len
        self.hidden_nodes = []
        self.init_connections()
                  
    def init_connections(self):
        for i in range(self.input_len):
            for j in range(self.input_len, self.output_len + self.input_len):
                self.connections.append(ConnectionTemplate(i, j, self.innovation_num))
                self.innovation_num += 1
                
    def add_node(self, in_node, out_node):
        for i, node in enumerate(self.hidden_nodes):
            if node.in_node == in_node and node.out_node == out_node:
                return node
        if in_node >= (self.input_len + self.output_len) and out_node >= (self.input_len + self.output_len):
            for node in self.hidden_nodes:
                if node.node_id == in_node:
                    in_found = True
                if node.node_id == out_node:
                    out_found = True
            if not(in_found and out_found):
                print("Invalid nodes")
                return None
        con1 = ConnectionTemplate(in_node, self.node_num, self.innovation_num)
        self.connections.append(con1)
        self.innovation_num += 1
        con2 = ConnectionTemplate(self.node_num, out_node, self.innovation_num)
        self.connections.append(con2)
        self.innovation_num += 1
        node_template = NodeTemplate(self.node_num, in_node, out_node, con1, con2)
        self.hidden_nodes.append(node_template)
        self.node_num += 1
        return node_template
    
    def add_connection(self, in_node, out_node):
        for con in self.connections:
            if con.in_node == in_node and con.out_node == out_node:
                return con.innovation
        in_found = False
        out_found = False
        if in_node >= (self.input_len + self.output_len) and out_node >= (self.input_len + self.output_len):
            for node in self.hidden_nodes:
                if node.node_id == in_node:
                    in_found = True
                if node.node_id == out_node:
                    out_found = True
            if not(in_found and out_found):
                print("Invalid nodes")
                return None
        con_temp = ConnectionTemplate(in_node, out_node, self.innovation_num)
        self.connections.append(con_temp)
        self.innovation_num += 1
        return con_temp
                
                
class ConnectionTemplate:
    def __init__(self, in_node, out_node, innovation):
        self.in_node = in_node
        self.out_node = out_node
        self.innovation = innovation

        
class NodeTemplate:
    def __init__(self, node_id, in_node, out_node, con1, con2):
        self.node_id = node_id
        self.in_node = in_node
        self.out_node = out_node
        self.con1 = con1
        self.con2 = con2


class Node:
    def __init__(self, node_id):
        self.node_id = node_id
        self.value = 0
        self.activated = 0
        self.activation = self.linear
        self.bias = 0
    
    def linear(self, x):
        return x
    
    def activate(self):
        self.activated = self.activation(self.value + self.bias)
    
class Connection:
    def __init__(self, node_in_id, node_out_id, node_in, node_out, weight, enabled, innovation):
        self.node_in_id = node_in_id
        self.node_out_id = node_out_id
        self.node_in = node_in
        self.node_out = node_out
        self.weight = weight
        self.enabled = enabled
        self.innovation = innovation    
    
class Genome:
    def __init__(self, template):
        self.template = template
        self.input_len = self.template.input_len
        self.output_len = self.template.output_len
#         self.biases = [0 for _ in range(self.input_len + self.output_len)]
        self.nodes = [Node(i) for i in range(self.input_len + self.output_len)]
        self.connections = self.init_connections()
    
    def init_connections(self):
        connections = []
        for i, con in enumerate(self.template.connections):
            weight = np.random.normal()
            connections.append(Connection(con.in_node, 
                                          con.out_node, 
                                          self.nodes[con.in_node], 
                                          self.nodes[con.out_node], 
                                          weight, 
                                          True, 
                                          con.innovation))
        return connections
    
    def add_node(self, in_node, out_node):
        if in_node == out_node:
            return None
        weight = np.random.normal()
        node_template = self.template.add_node(in_node, out_node)
        if node_template is None:
            return None
        in_node_obj = None
        out_node_obj = None
        for nd in self.nodes:
            if node_template.node_id == nd.node_id:
                print("This node has been already added")
                return None
            if nd.node_id == in_node:
                in_node_obj = nd
            if nd.node_id == out_node:
                out_node_obj = nd
            
        node = Node(node_template.node_id)
        self.nodes.append(node)
#         self.biases.append(0)
        self.connections.append(Connection(node_template.con1.in_node, 
                                           node_template.con1.out_node,
                                           in_node_obj,
                                           node,
                                           weight, 
                                           True, 
                                           node_template.con1.innovation))
        self.connections.append(Connection(node_template.con2.in_node, 
                                           node_template.con2.out_node,
                                           node,
                                           out_node_obj,
                                           weight, 
                                           True, 
                                           node_template.con2.innovation))
    
    def add_connection(self, in_node, out_node):
        con_template = self.template.add_connection(in_node, out_node)
        for con in self.connections:
            if con.innovation == con_template.innovation:
                print("This connection has been already created")
                return None
        in_node_obj = None
        out_node_obj = None
        for nd in self.nodes:
            if nd.node_id == in_node:
                in_node_obj = nd
            if nd.node_id == out_node:
                out_node_obj = nd
        weight = np.random.normal()
        self.connections.append(Connection(in_node, 
                                           out_node,
                                           in_node_obj,
                                           out_node_obj,
                                           weight, 
                                           True, 
                                           con_template.innovation))
        
     
    def clear_nodes(self):
        for i in range(self.input_len, len(self.nodes)):
            self.nodes[i].value = 0
            
    def forward(self):
        for con in self.connections:
            con.node_in.activate() 
            con.node_out.value += con.node_in.activated
            
    def set_input(self, x):
        if len(x) != self.input_len:
            print("Invalid input size")
            return None
        for i in range(self.input_len):
            self.nodes[i].value = x[i]
            
              
class NEAT:
    def __init__(self, input_len, output_len, n_genomes):
        self.input_len = input_len
        self.output_len = output_len
        self.n_genomes = n_genomes
        self.genome_template = GenomeTemplate(self.input_len, self.output_len)
        self.genomes = [Genome(self.genome_template) for _ in range(n_genomes)]
        

        
    
    
    

In [129]:
neat = NEAT(3, 1, 10)

In [130]:
neat.genomes[0].add_node(0, 3)

In [131]:
neat.genomes[0].add_connection(1, 4)

In [132]:
neat.genomes[0].set_input([1, 2, 3])

In [139]:
neat.genomes[0].forward()

In [140]:
neat.genomes[0].nodes[3].value

30

In [141]:
neat.genomes[0].nodes[0].value

1